The dataset is provided by GroupLens and can be downloaded from https://grouplens.org/datasets/movielens/ it contains the following files(movies.csv,ratings.csv)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
moviesData=pd.read_csv('Data/movies.csv')
ratingsData=pd.read_csv('Data/ratings.csv')

In [3]:
moviesData.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratingsData.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
ratingsData['timestamp'] = pd.to_datetime(ratingsData['timestamp'], unit='s')
ratingsData.columns = ['userId', 'movieId','rating' ,'timing']
ratingsData[:5]

,userId,movieId,rating,timing
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51


In [ ]:

moviesData[:5]

In [6]:
import pyodbc # a phyton library used to access and use Sql on python
import sys# used to manipulate different parts of the Python Runtime Environment
class PostgreSQLconnection(object):
    def __init__(self, DATABASE,UID,PWD,SERVER):
        self.db_connection=self.establish_db_connection(DATABASE,UID,PWD,SERVER)
        self.cursor = self.db_connection.cursor()
    def close_db_connecion(self):
        self.cursor.close()
        self.db_connection.close()
    def establish_db_connection(self,DATABASE,UID,PWD,SERVER):
        try:
                conn_str  = (
                    "DRIVER={Devart ODBC Driver for MYSQL};"
                    "DATABASE="+DATABASE+";"
                    "UID="+UID+";"
                    "PWD="+PWD+";"
                    "SERVER="+SERVER+";"
                    "PORT=5432;"
                )
                connection = pyodbc.connect(conn_str)
                print("Successfully connected to database.")
                return connection
        
        except:
                # Couldn't connect to database. Print error message with more details
                sys.exit('Failed to connect to database:')
    def create_table(self,sqlstatement,Table):
        self.cursor.execute(sqlstatement)
        print("Table created successfully with name"+Table)
        self.db_connection.commit()
        
    def run_command(self,command):
        self.cursor.execute(command)
        print("successfull execution of command")
        self.db_connection.commit()
        return
    def run_query(self,query):
        return pd.read_sql_query(query,con=self.db_connection)
db=PostgreSQLconnection("sample","postgres","Passwordme","localhost")

Successfully connected to database.


In [7]:
#sql statement to create table
sqlstatement='''CREATE TABLE Movies(
              movieId INT,
              title text,
              genres text,
              PRIMARY KEY(movieId)
              )'''
db.create_table(sqlstatement, ' Movies')
#sql statement to create table
sqlstatement='''CREATE TABLE Rating(
              userId INT,
              movieId INT,
              rating REAL,
              timing TIMESTAMP,
              PRIMARY KEY(movieId,userId),
              FOREIGN KEY(movieId) 
                  REFERENCES Movies(movieId)
              )'''
db.create_table(sqlstatement, ' Ratings')

Table created successfully with name Movies
Table created successfully with name Ratings


In [8]:
cursor = db.cursor
for row in moviesData.itertuples():
    cursor.execute('''
                        INSERT INTO Movies(movieId,title,genres)
                        VALUES (?,?,?)
                        ''',
                        row.movieId,
                        row.title,
                        row.genres
                  )
db.db_connection.commit()
print("successfully inserted data from DataFrame into table")
for row in ratingsData.itertuples():
    cursor.execute('''
                        INSERT INTO Rating(userId,movieId,rating,timing)
                        VALUES (?,?,?,?)
                        ''',
                        row.userId,
                        row.movieId,
                        row.rating,
                        row.timing
                  )
db.db_connection.commit()
print("successfully inserted data from DataFrame into table")

successfully inserted data from DataFrame into table
successfully inserted data from DataFrame into table
